In [1]:
from rdflib import Graph, Literal, Namespace
from rdflib.namespace import RDFS
import hashlib

g = Graph()

terms = Namespace("http://related-terms.io/terms/")
termsv = Namespace("http://related-terms.io/vocab#")

relaredToPred = termsv.relatedTo
labelPred = RDFS.label

def md5_hash(t):
    result = hashlib.md5(t.encode())
    return result.hexdigest()

def add_terms(t1, t2):
    t1 = t1.lower()
    t2 = t2.lower()
    t1h = md5_hash(t1)
    t2h = md5_hash(t2)
    g.add((terms[t1h], relaredToPred, terms[t2h]))
    g.add((terms[t2h], relaredToPred, terms[t1h]))
    g.add((terms[t1h], labelPred, Literal(t1)))
    g.add((terms[t2h], labelPred, Literal(t2)))    


In [2]:
files = ['conceptnet-english.csv', 'mesh-related-terms.txt']
for file in files:
    with open(file) as f:
        for l in f:
            l = l.strip().lower().split('|||')
            add_terms(l[0], l[1])
            
g.serialize(destination="related-terms.nt", format='ntriples')

In [3]:
!aws s3 cp related-terms.nt s3://osedemah-mesh-rdf/related-terms.nt

upload: ./related-terms.nt to s3://osedemah-mesh-rdf/related-terms.nt


In [4]:
!curl -X POST \
    -H 'Content-Type: application/json' \
    https://database-1.cluster-cclypxvaeodz.us-west-2.neptune.amazonaws.com:8182/loader -d '\
    {\
      "source" : "s3://osedemah-mesh-rdf/related-terms.nt",\
      "format" : "ntriples",\
      "iamRoleArn" : "arn:aws:iam::812630329048:role/NeptuneS3",\
      "region" : "us-west-2",\
      "failOnError" : "FALSE",\
      "parallelism" : "MEDIUM",\
      "updateSingleCardinalityProperties" : "FALSE",\
      "queueRequest" : "TRUE"\
    }'

{
    "status" : "200 OK",
    "payload" : {
        "loadId" : "cf5b9788-696b-40fb-94a3-5a37dc8cf23a"
    }
}

In [18]:
!curl https://database-1.cluster-cclypxvaeodz.us-west-2.neptune.amazonaws.com:8182/loader/cf5b9788-696b-40fb-94a3-5a37dc8cf23a

{
    "status" : "200 OK",
    "payload" : {
        "feedCount" : [
            {
                "LOAD_COMPLETED" : 1
            }
        ],
        "overallStatus" : {
            "fullUri" : "s3://osedemah-mesh-rdf/related-terms.nt",
            "runNumber" : 1,
            "retryNumber" : 0,
            "status" : "LOAD_COMPLETED",
            "totalTimeSpent" : 495,
            "startTime" : 1689535651,
            "totalRecords" : 7423261,
            "totalDuplicates" : 0,
            "parsingErrors" : 0,
            "datatypeMismatchErrors" : 0,
            "insertErrors" : 0
        }
    }
}

In [1]:
!wget https://s3.amazonaws.com/aws-neptune-customer-samples/neptune-export/bin/neptune-export.jar

--2023-08-09 01:11:28--  https://s3.amazonaws.com/aws-neptune-customer-samples/neptune-export/bin/neptune-export.jar
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.71.182, 52.216.222.144, 52.216.230.61, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.71.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74647740 (71M) [application/java-archive]
Saving to: ‘neptune-export.jar’

100%[======================================>] 74,647,740  35.4MB/s   in 2.0s   

2023-08-09 01:11:30 (35.4 MB/s) - ‘neptune-export.jar’ saved [74647740/74647740]



In [5]:
!java -jar neptune-export.jar nesvc --json '{"command":"export-rdf","outputS3Path":"s3://osedemah-mesh-rdf/neptune-export","params":{"endpoint":"database-1.cluster-cclypxvaeodz.us-west-2.neptune.amazonaws.com"}}'

neptune-export.jar: [buildVersion='1.0.6', buildTime='2023-07-31T23:08:33+0000', commitId='f4ee9d1b1c10ff69cdb11534850940f5ca838918', commitTime='2023-07-31T23:08:00+0000']
cmd                       : export-rdf
params                    : {
  "endpoint" : "database-1.cluster-cclypxvaeodz.us-west-2.neptune.amazonaws.com"
}
outputS3Path              : s3://osedemah-mesh-rdf/neptune-export
createExportSubdirectory  : true
overwriteExisting         : false
uploadToS3OnError         : true
configFileS3Path          : 
queriesFileS3Path         : 
completionFileS3Path      : 
s3Region                  : 
sseKmsKeyId               : 
completionFilePayload     : { }
additionalParams          : { }
maxFileDescriptorCount    : 10000
revised cmd           : export-rdf --endpoint 'database-1.cluster-cclypxvaeodz.us-west-2.neptune.amazonaws.com'
[main] INFO com.amazonaws.services.neptune.export.NeptuneExportService - Args after service init: export-rdf --endpoint database-1.cluster-cclypxvaeodz.us

In [6]:
!rm -rf exports

In [7]:
!ls

conceptnet-english.csv	neptune-export.jar   related-terms.nt
create-rdf-dump.ipynb	query.ipynb
mesh-related-terms.txt	related_terms.ipynb
